# Downloading Data from Bank of Korea using API

### week 3-2

We have learn [how to download data of interest using Bank of Korea API](https://github.com/JKang918/ExchangeRatePrediction/blob/main/3.%20Bank%20of%20Korea%20API.ipynb).\
Let's expand on that and learn how to build functions to download multiple economic data in an effective way.

Refer to [BOK development specification](https://ecos.bok.or.kr/api/#/DevGuide/DevSpeciflcation) for more inforamtion about BOK API.


> *Note 1. source code, exmaples: KAIST Business school, Prof. Jaehoon Lee*\
> *Note 2. The website is optimized for Korean speakers. If you do not speak Korean, I recommend you to still go for the Korean website and use in-browser translation services like Google translate.* \
> *Note 3. The codes in this file was tested in July 2024. However, later updates in BOK system might be needed to be checked beforehand.* 

In [1]:
import requests
import pandas as pd

from pandas.tseries.offsets import MonthEnd, YearEnd

In [2]:
BOK_API_URL = 'https://ecos.bok.or.kr/api'
BOK_API_KEY = 'IDI4QC444EYRWRBBQ8JN' #individual specific API key

In [4]:
#function structured specifically for BOK API system
def fetch_bok_data(service_code, stat_code = None, item = None):
    url = f'{BOK_API_URL}/{service_code}/{BOK_API_KEY}/json/en/1/100000' #10000 is sufficiently larget, but arbitrary number

    if item is not None:
        url += "/{}/{}/{}/{}/{}".format(

            item['STAT_CODE'], 
            item['CYCLE'],
            item['START_TIME'],
            item['END_TIME'],
            item['ITEM_CODE'],
        )

        #BOK API development guide - item code itself is stratified -> There can be multiple codes 
        if type(item['ITEM_CODE2']) is str:
            url += f"/{item['ITEM_CODE2']}"
        
    elif stat_code:
        url += f'/{stat_code}' 
    
    print(url)
    resp = requests.get(url)
    results = resp.json()
    rows = results[service_code]['row']
    df = pd.DataFrame(rows)

    return df

To recap, BOK API is constructed as below.


1. **[StatisticTableList]** Identify data of your interest from the list of available datasets. 
    - In this step, we obtain the code name for the dataset we require. (e.g., 731Y001)


2. **[StatisticItemList]** Identify items within the dataset.
   - Here, we obtain the name of item we require


3. **[StatisticSearch]** Fectch item values
   - We fetch data of our interest.


### Collect Item Codes

> First, we need to identify the code names for data sets we require (*StatisticTableList*)

In [5]:
#StatisticTableList: list names of data sets BOK offers
stat_codes = fetch_bok_data('StatisticTableList')

https://ecos.bok.or.kr/api/StatisticTableList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000


> Among the list of data set names, you can search names containing certain terms like below:

In [6]:
#find data set name including 'consumer'
idx = stat_codes['STAT_NAME'].str.lower().str.find('arbitraged') >= 0
stat_codes.loc[idx]

,P_STAT_CODE,STAT_CODE,STAT_NAME,CYCLE,SRCH_YN,ORG_NAME
568,0000000178,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,D,Y,Seoul Money Broker Service
572,0000000182,731Y004,3.1.2.1. Arbitraged Rates of Major Currencies ...,M,Y,Bank of Korea


> Here, we assume you found all table codes of interest following above process.

In [7]:
#assume we found target data sets like above.
TARGET_STAT_CODES = [
    '731Y001',  # 3.1.1.1. Arbitraged Rates of Major Currencies Against Won
    '902Y008',  # 9.1.2.2. International Consumer Price indices
    '902Y009',  # 9.1.3.1. Internatioanl Current Account, Total, Net, US Dollars
    '902Y015',  # 9.1.4.1. International Growth Rate of GDP
    '902Y016',  # 9.1.4.2. GDP
    '902Y023',  # 9.1.6.1. Financial Market Key Indicators
    '817Y002',  # 1.3.2.1. Market Interest Rates(Daily)
    '732Y001',  # 3.5. International Reserves
]

> As we now know the code name of data sets we require, we can proceed to download items in each data set. (*StatisticItemList*)

In [8]:
#Append to an array, the eight tables of interest
dfs = []

for stat_code in TARGET_STAT_CODES:
    df = fetch_bok_data('StatisticItemList', stat_code = stat_code)

    dfs.append(df)

https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/731Y001
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y008
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y009
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y015
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y016
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y023
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/817Y002
https://ecos.bok.or.kr/api/StatisticItemList/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/732Y001


In [9]:
#for example,
dfs[2]

,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,P_ITEM_CODE,P_ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT,UNIT_NAME,WEIGHT
0,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",KR,"Korea, Republic Of",None,None,A,1976,2023,48,Mil.U$,None
1,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",KR,"Korea, Republic Of",None,None,Q,1976Q1,2024Q1,193,Mil.U$,None
2,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",AU,Australia,None,None,A,1989,2023,35,Mil.U$,None
3,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",AU,Australia,None,None,Q,1989Q1,2024Q1,141,Mil.U$,None
4,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",AT,Austria,None,None,A,2005,2023,19,Mil.U$,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",TR,Turkiye,None,None,Q,1984Q1,2024Q1,161,Mil.U$,None
76,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",GB,United Kingdom,None,None,A,1970,2023,54,Mil.U$,None
77,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",GB,United Kingdom,None,None,Q,1970Q1,2024Q1,217,Mil.U$,None
78,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",Group1,"Current Account, Total, Net, US Dollars",US,United States,None,None,A,1970,2023,54,Mil.U$,None


In [10]:
#concat the eight tables vertically
df_item_codes = pd.concat(dfs, axis = 0)

In [11]:
df_item_codes

,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,P_ITEM_CODE,P_ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT,UNIT_NAME,WEIGHT
0,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,Group1,Selection of Currencies,0000001,Won per United States Dollar(Basic Exchange Rate),None,None,D,19640504,20240809,16998,Won,None
1,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,Group1,Selection of Currencies,0000053,Won per Yuan(Basic Exchange Rate),None,None,D,20160104,20240809,2125,Won,None
2,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,Group1,Selection of Currencies,0000002,Won per Japanese Yen(100Yen),None,None,D,19770401,20240809,13150,Won,None
3,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,Group1,Selection of Currencies,0000003,Won per Euro,None,None,D,19940411,20240809,8112,Won,None
4,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,Group1,Selection of Currencies,0000004,Won per German Mark,None,None,D,19640504,20011231,11170,Won,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,732Y001,3.5. International Reserves,Group1,Selection of Items,04,Foreign Currency Reserves,99,Total,M,197101,202407,643,Thou.U$,None
11,732Y001,3.5. International Reserves,Group1,Selection of Items,04,Foreign Currency Reserves,99,Total,Q,1971Q1,2024Q2,214,Thou.U$,None
12,732Y001,3.5. International Reserves,Group1,Selection of Items,99,Total,None,None,A,1960,2023,64,Thou.U$,None
13,732Y001,3.5. International Reserves,Group1,Selection of Items,99,Total,None,None,M,197101,202407,643,Thou.U$,None


In [12]:
#write it to excel
df_item_codes.to_excel('data/bok_item_codes.xlsx', index = False)

In [13]:
df_item_codes = df_item_codes.set_index(['STAT_CODE', 'ITEM_CODE', 'CYCLE'])

In [14]:
df_item_codes

STAT_NAME  \
STAT_CODE ITEM_CODE CYCLE                                                      
731Y001   0000001   D      3.1.1.1. Arbitraged Rates of Major Currencies ...   
          0000053   D      3.1.1.1. Arbitraged Rates of Major Currencies ...   
          0000002   D      3.1.1.1. Arbitraged Rates of Major Currencies ...   
          0000003   D      3.1.1.1. Arbitraged Rates of Major Currencies ...   
          0000004   D      3.1.1.1. Arbitraged Rates of Major Currencies ...   
...                                                                      ...   
732Y001   04        M                            3.5. International Reserves   
                    Q                            3.5. International Reserves   
          99        A                            3.5. International Reserves   
                    M                            3.5. International Reserves   
                    Q                            3.5. International Reserves   

                          GRP_CODE                 GRP_NAME  \
STAT_CODE ITEM_CODE CYCLE                                     
731Y001   0000001   D       Group1  Selection of Currencies   
          0000053   D       Group1  Selection of Currencies   
          0000002   D       Group1  Selection of Currencies   
          0000003   D       Group1  Selection of Currencies   
          0000004   D       Group1  Selection of Currencies   
...                            ...                      ...   
732Y001   04        M       Group1       Selection of Items   
                    Q       Group1       Selection of Items   
          99        A       Group1       Selection of Items   
                    M       Group1       Selection of Items   
                    Q       Group1       Selection of Items   

                                                                   ITEM_NAME  \
STAT_CODE ITEM_CODE CYCLE                                                      
731Y001   0000001   D      Won per United States Dollar(Basic Exchange Rate)   
          0000053   D                      Won per Yuan(Basic Exchange Rate)   
          0000002   D                           Won per Japanese Yen(100Yen)   
          0000003   D                                           Won per Euro   
          0000004   D                                    Won per German Mark   
...                                                                      ...   
732Y001   04        M                              Foreign Currency Reserves   
                    Q                              Foreign Currency Reserves   
          99        A                                                  Total   
                    M                                                  Total   
                    Q                                                  Total   

                          P_ITEM_CODE P_ITEM_NAME START_TIME  END_TIME  \
STAT_CODE ITEM_CODE CYCLE                                                
731Y001   0000001   D            None        None   19640504  20240809   
          0000053   D            None        None   20160104  20240809   
          0000002   D            None        None   19770401  20240809   
          0000003   D            None        None   19940411  20240809   
          0000004   D            None        None   19640504  20011231   
...                               ...         ...        ...       ...   
732Y001   04        M              99       Total     197101    202407   
                    Q              99       Total     1971Q1    2024Q2   
          99        A            None        None       1960      2023   
                    M            None        None     197101    202407   
                    Q            None        None     1971Q1    2024Q2   

                           DATA_CNT UNIT_NAME WEIGHT  
STAT_CODE ITEM_CODE CYCLE                             
731Y001   0000001   D         16998       Won   None  
          0000053   D          2125  

### Download Data Values

Now that we have item codes, we can download data values. 

In [19]:
data_download_list = pd.read_excel('data/bok_data_download_list.xlsx')

In [20]:
data_download_list

,name,STAT_CODE,STAT_NAME,ITEM_CODE,ITEM_CODE2,ITEM_NAME,CYCLE,UNIT_NAME
0,krw_usd_xr,731Y001,3.1.1.1. Arbitraged Rates of Major Currencies ...,0000001,NaN,Won per United States Dollar(Basic Exchange Rate),D,Won
1,cpi_korea,902Y008,9.1.2.2. International Consumer Price indices,KR,NaN,"Korea, Republic Of",M,2010=100
2,cpi_us,902Y008,9.1.2.2. International Consumer Price indices,US,NaN,United States,M,2010=100
3,current_account_korea,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",KR,NaN,"Korea, Republic Of",Q,Mil.U$
4,current_account_us,902Y009,"9.1.3.1. Internatioanl Current Account, Total,...",US,NaN,United States,Q,Mil.U$
5,gdp_growth_korea,902Y015,9.1.4.1. International Growth Rate of GDP,KOR,NaN,"Korea, Republic Of",Q,%
6,gdp_growth_us,902Y015,9.1.4.1. International Growth Rate of GDP,USA,NaN,United States,Q,%
7,gdp_korea,902Y016,9.1.4.2. GDP,KOR,NaN,"Korea, Republic Of",A,Mil.U$
8,gdp_us,902Y016,9.1.4.2. GDP,USA,NaN,United States,A,Mil.U$
9,foreign_reserves,732Y001,3.5. International Reserves,99,NaN,Total,M,Thou.U$


In [21]:
#iterate over rows
for idx, row in data_download_list.iterrows():
    name = row['name']

    print(f"Downloading {name} ...")
    idx = (row.STAT_CODE, row.ITEM_CODE, row.CYCLE)
    item = df_item_codes.loc[idx]

    #merge downloaded data and preparaed excel file (itemlists of interest)
    item = {**item, **row}

    #from items of interest, fetch data
    df = fetch_bok_data('StatisticSearch', item = item)
    df = df[['TIME', 'DATA_VALUE']]

    #for timestamp

    #daily data - 8 digit date
    if row['CYCLE'] == 'D':
        df['date'] = pd.to_datetime(df['TIME'])

    #monthly data - 6 digit date
    elif row['CYCLE'] == 'M':
        df['date'] = pd.to_datetime(df['TIME'], format = '%Y%m') + MonthEnd()

    #quarterly data - yyyyQx
    elif row['CYCLE'] == 'Q':
        df['year'] = df['TIME'].str[:4]
        df['month'] = df['TIME'].str[-1].astype(int) * 3
        df['day'] = 1
        df['date'] = pd.to_datetime(df[['year', 'month', 'day']]) + MonthEnd()

    #annual data - 4 digit date
    elif row['CYCLE'] == 'A':
        df['date'] = pd.to_datetime(df['TIME']) + YearEnd()
    
    else:
        raise RuntimeError()
    
    
    df = df[['date', 'DATA_VALUE']]
    df = df.rename(columns = {
        'DATA_VALUE' : name,
    })
    df[name] = df[name].astype(float)
    df.to_excel(f"data/{name}.xlsx", index = False)
    
    print()

print('** Completed !! **')

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/731Y001/D/19640504/20240809/0000001

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y008/M/195108/202406/KR

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y008/M/195501/202406/US

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y009/Q/1976Q1/2024Q1/KR

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y009/Q/1973Q1/2024Q1/US

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y015/Q/1960Q2/2024Q2/KOR

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y015/Q/1947Q2/2024Q2/USA

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y016/A/1953/2023/KOR

https://ecos.bok.or.kr/api/StatisticSearch/IDI4QC444EYRWRBBQ8JN/json/en/1/100000/902Y016/A/1970/2023/USA

https://ecos.b

> You can find 12 excel files downloaded in your data folder.\
> We are going to use these macroeconomic data for our forecasting model.